# Echo Msk

In [1]:
import urllib.request
import json
import pandas as pd
from token_key import k
import ssl
import re
import tqdm
import random
regex = re.compile("[А-ЯЙ]{3,}") 

In [2]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [3]:
def readUrl(url, key=k):
    url = url + k
    content = urllib.request.urlopen(url, context=ctx)
    decoded_content = content.read().decode("utf-8")
    data = json.loads(decoded_content)
    return data

## Cобираем id всех постов со страницы

In [4]:
data = readUrl("https://api.vk.com/method/wall.get?owner_id=-60556804&count=100&v=5.95&access_token=")
post_ids = {data["response"]["items"][i]["id"]:data["response"]["items"][i]["text"] for i in range(len(data["response"]["items"]))}
offset = 101
while offset < 50002:
    data = readUrl("https://api.vk.com/method/wall.get?owner_id=-60556804&offset={}&count=100&v=5.95&access_token=".format(offset))
    new_post_ids = {data["response"]["items"][i]["id"]:data["response"]["items"][i]["text"] for i in range(len(data["response"]["items"]))}
    post_ids.update(new_post_ids)
    offset += 100

In [5]:
keys = list(post_ids.keys())

In [7]:
len(keys)

47099

In [6]:
result_entries = []
for id_ in keys:
    offset = 0
    while offset < 502: 
        data = readUrl("https://api.vk.com/method/wall.getComments?owner_id=-60556804&post_id={}&offset={}&count=100&v=6.7&access_token=".format(id_,offset))
        try:
            data = pd.DataFrame(data["response"]["items"], columns = ["from_id", "post_id", "text", "id"])
            data_list = data.values.tolist()
            lines = [i[2] for i in data_list]
            offset += 100
            for i, line in enumerate(lines):
                try:
                    if re.search(regex, line):
                        result_entries.append(data_list[i])
                except TypeError:
                    pass
            
            comment_ids = [i[3] for i in data_list]
            for com_id in comment_ids:
                comment_data = readUrl("https://api.vk.com/method/wall.getComments?owner_id=-60556804&comment_id={}&count=100&v=6.7&access_token=".format(com_id))
                comment_data = pd.DataFrame(comment_data["response"]["items"], columns = ["from_id", "post_id", "text", "id"])
                comment_data_list = comment_data.values.tolist()
                lines2 = [i[2] for i in comment_data_list]
                for i, line in enumerate(lines2):
                    try:
                        if re.search(regex, line):
                            result_entries.append(comment_data_list[i])
                    except TypeError:
                        pass

        except KeyError:
            continue

KeyboardInterrupt: 

In [13]:
df_res = pd.DataFrame(result_entries, columns = ["from_id", "post_id", "comment_text", "id"])
df_res.count()

from_id         24232
post_id         24232
comment_text    24232
id              24232
dtype: int64

In [14]:
df_res.head()

,from_id,post_id,comment_text,id
0,397494426.0,3498493.0,Был я в 1999 году в командировке в женской кол...,3498509
1,304989159.0,3498351.0,"[id545055378|Seyfullah], потому , что даже МИД...",3498440
2,477440631.0,3498341.0,ОДНОЗНАЧНО...,3498490
3,1080747.0,3498293.0,Печально. Страшно ЖЫЫДЬ!,3498296
4,542471717.0,3498293.0,Шаверма-патруль как пример видео о социально-з...,3498322


In [15]:
df_res["first_name"] = None 
df_res["last_name"] = None 
df_res["sex"] = None
df_res["bdate"] = None 
df_res["city"] = None
df_res.head()

,from_id,post_id,comment_text,id,first_name,last_name,sex,bdate,city
0,397494426.0,3498493.0,Был я в 1999 году в командировке в женской кол...,3498509,None,None,None,None,None
1,304989159.0,3498351.0,"[id545055378|Seyfullah], потому , что даже МИД...",3498440,None,None,None,None,None
2,477440631.0,3498341.0,ОДНОЗНАЧНО...,3498490,None,None,None,None,None
3,1080747.0,3498293.0,Печально. Страшно ЖЫЫДЬ!,3498296,None,None,None,None,None
4,542471717.0,3498293.0,Шаверма-патруль как пример видео о социально-з...,3498322,None,None,None,None,None


In [16]:
for i in df_res.index:
    user_id = int(df_res.from_id[i])
    data = readUrl("https://api.vk.com/method/users.get?user_ids={}&fields=bdate,sex,city&v=5.95&access_token=".format(user_id))
    try:
        l = list(data["response"])[0]
        l["city"] = l["city"]["title"]
    except KeyError:
        continue
    df_res.loc[i, "first_name"] = l.get("first_name", None)
    df_res.loc[i, "last_name"] = l.get("last_name", None)
    df_res.loc[i, "sex"] = l.get("sex", None)
    df_res.loc[i,"bdate"] = l.get("bdate", None)
    df_res.loc[i,"city"] = l.get("city", None)
    
# работает минут 20

In [17]:
df_res.post_id.nunique()

5559

In [18]:
df_res.head()

,from_id,post_id,comment_text,id,first_name,last_name,sex,bdate,city
0,397494426.0,3498493.0,Был я в 1999 году в командировке в женской кол...,3498509,None,None,None,None,None
1,304989159.0,3498351.0,"[id545055378|Seyfullah], потому , что даже МИД...",3498440,None,None,None,None,None
2,477440631.0,3498341.0,ОДНОЗНАЧНО...,3498490,Vitaly,Domansky,2,4.4.1959,Saint Petersburg
3,1080747.0,3498293.0,Печально. Страшно ЖЫЫДЬ!,3498296,Denis,Cherednichenko,2,15.12.1977,Saint Petersburg
4,542471717.0,3498293.0,Шаверма-патруль как пример видео о социально-з...,3498322,Gennady,Zhuravlyov,2,27.4.1981,Saint Petersburg


In [19]:
df_test = df_res.dropna(subset=["sex"]) 
df_test.drop_duplicates(subset=["from_id"], keep='last', inplace=True)

c:\users\1cc38\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [20]:
df_test.head()

,from_id,post_id,comment_text,id,first_name,last_name,sex,bdate,city
4,542471717.0,3498293.0,Шаверма-патруль как пример видео о социально-з...,3498322,Gennady,Zhuravlyov,2,27.4.1981,Saint Petersburg
6,238348462.0,3498293.0,АААААААААААААААхуенно. Это такой дичайший ФАРС...,3498479,Andrey,Vasyutin,2,21.10,Moscow
25,3514876.0,3498024.0,Перепутали Крым и Севастополь с ДНР и ЛНР.,3498160,Alexander,Smirnov,2,17.2,Saint Petersburg
83,218309623.0,3497119.0,"[id509266047|Марина], ЕСЛИ ВЫ ТРОЛЛЬ, ТО ХОТЬ ...",3497937,Lola,Khan,1,None,Moscow
102,140289491.0,3496841.0,"Так, диванные эксперты, нужна ваша критика!\n\...",3496909,Zaur,Medvedev,2,None,Saint Petersburg


In [21]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2421 entries, 4 to 24229
Data columns (total 9 columns):
from_id         2421 non-null float64
post_id         2421 non-null float64
comment_text    2421 non-null object
id              2421 non-null int64
first_name      2421 non-null object
last_name       2421 non-null object
sex             2421 non-null object
bdate           1743 non-null object
city            2421 non-null object
dtypes: float64(2), int64(1), object(6)
memory usage: 189.1+ KB


In [22]:
df_test.sex.value_counts()

2    1881
1     539
0       1
Name: sex, dtype: int64

In [23]:
df_test.to_csv("capitalization-echo-wall-2.csv", index=False)